<a href="https://colab.research.google.com/github/ramirezramiro/token-address-fetcher/blob/main/token-address-fetcher-bridged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
import os

# Base URL for CoinMarketCap API
CMC_API_LINK = 'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing'

def fetch_platform_names(limit=5000):
    """
    Fetches cryptocurrency data and returns a sorted list of unique platform names.
    """
    params = {
        'start': 1,
        'limit': limit,
        'sortBy': 'market_cap',
        'sortType': 'desc',
        'convert': 'USD',
        'cryptoType': 'all',
        'tagType': 'all',
        'aux': 'platform,tags' # Requesting 'tags' in aux
    }

    unique_platform_names = set()
    print("Fetching available token platform names from CoinMarketCap API...")

    with requests.Session() as s:
        s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        try:
            res = s.get(CMC_API_LINK, params=params, timeout=30)
            res.raise_for_status()
            data = res.json()

            if data and 'data' in data and 'cryptoCurrencyList' in data['data']:
                for item in data['data']['cryptoCurrencyList']:
                    platform = item.get('platform')
                    if platform and 'name' in platform:
                        unique_platform_names.add(platform['name'])
            else:
                print("Warning: No cryptocurrency data found in the response or unexpected data structure.")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching platform names: {e}")
            print("Please check your internet connection or try again later.")

    return sorted(list(unique_platform_names))

def is_bridged_or_wrapped(tags):
    """
    Checks if a token's tags indicate it is a bridged or wrapped asset.
    """
    if not tags:
        return False

    # Common tags indicating a wrapped or bridged token
    # These may vary slightly based on CMC's data
    bridged_tags = [
        'wrapped', 'bridge', 'cross-chain', 'pegged', 'tokenized',
        'erc20', 'bep20', 'trc20', 'spl' # Specific standards implying a wrapped version if platform differs from original
    ]

    for tag_item in tags:
        if isinstance(tag_item, dict) and 'slug' in tag_item: # Check if it's a dict and has 'slug'
            tag_slug = tag_item['slug'].lower()
            if any(bt in tag_slug for bt in bridged_tags):
                return True
        elif isinstance(tag_item, str): # Fallback for older/simpler tag structures
            if any(bt in tag_item.lower() for bt in bridged_tags):
                return True
    return False

def fetch_and_export_token_data(selected_platform_name):
    """
    Fetches cryptocurrencies for a given platform name, prints them, and exports them to a CSV.
    Includes a check for bridged/wrapped status.
    """
    params = {
        'start': 1,
        'limit': 1000, # Increased limit to ensure more tokens are fetched for the chosen platform
        'sortBy': 'market_cap',
        'sortType': 'desc',
        'convert': 'USD,BTC,ETH',
        'cryptoType': 'all',
        'tagType': 'all',
        'audited': 'false',
        # Ensure 'tags' is included in aux for bridged/wrapped check
        'aux': 'ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,tags,platform,max_supply,circulating_supply,total_supply,volume_7d,volume_30d'
    }

    safe_platform_name = selected_platform_name.replace(" ", "_").replace("/", "_").replace("\\", "_").lower()
    csv_filename = f"{safe_platform_name}_tokens.csv"

    # Add a new column for "Is Bridged/Wrapped?"
    fieldnames = ['CMC Rank', f'Coin Name ({selected_platform_name}-based)', 'Token Mint Address', 'Is Bridged/Wrapped?']

    tokens_data = []

    print(f"\nFetching '{selected_platform_name}'-based cryptocurrencies...")

    with requests.Session() as s:
        s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        try:
            res = s.get(CMC_API_LINK, params=params, timeout=30)
            res.raise_for_status()
            data = res.json()

            if data and 'data' in data and 'cryptoCurrencyList' in data['data']:
                print(f"\n--- Live Output for {selected_platform_name} Tokens ---")
                print(f"{'CMC Rank':<9} | {'Coin Name':<28} | {'Token Mint Address':<45} | {'Bridged/Wrapped?'}")
                print(f"--------------------------------------------------------------------------------------------------------------------")
                for item in data['data']['cryptoCurrencyList']:
                    platform = item.get('platform')

                    if platform and platform.get('name') == selected_platform_name:
                        rank = item['cmcRank']
                        coin_name = item['name']
                        token_mint_address = platform.get('token_address') or platform.get('contract_address')

                        display_address = token_mint_address if token_mint_address else 'N/A (Not Available via CMC API)'

                        # Check for bridged/wrapped status based on tags
                        is_bridged = "Yes" if is_bridged_or_wrapped(item.get('tags')) else "No"

                        # Print to console
                        print(f"{rank:<9} | {coin_name:<28} | {display_address:<45} | {is_bridged}")

                        # Add data to our list for CSV export
                        tokens_data.append({
                            'CMC Rank': rank,
                            f'Coin Name ({selected_platform_name}-based)': coin_name,
                            'Token Mint Address': display_address,
                            'Is Bridged/Wrapped?': is_bridged
                        })
                print(f"--------------------------------------------------------------------------------------------------------------------")
            else:
                print("No cryptocurrency data found in the response for this platform or unexpected data structure.")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching token data for '{selected_platform_name}': {e}")
            print("Please check your internet connection or try again later.")
            return

    if tokens_data:
        try:
            with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(tokens_data)
            print(f"\nSuccessfully exported {len(tokens_data)} '{selected_platform_name}'-based cryptocurrencies to '{csv_filename}'")
            print(f"File saved in: {os.path.abspath(csv_filename)}")
        except IOError:
            print(f"I/O error: Could not write to file '{csv_filename}'")
    else:
        print(f"No '{selected_platform_name}'-based cryptocurrencies found to export.")

def main():
    """
    Main function to orchestrate the script's execution.
    """
    available_platforms = fetch_platform_names()

    if not available_platforms:
        print("Could not retrieve available platform names. Exiting.")
        return

    print("\nAvailable Token Platform Names:")
    for i, name in enumerate(available_platforms):
        print(f"  {i+1}. {name}")

    selected_platform = None
    while selected_platform is None:
        user_input = input("\nEnter the exact name of the token platform you want to query (or 'exit' to quit): ").strip()
        if user_input.lower() == 'exit':
            print("Exiting script.")
            return

        if user_input in available_platforms:
            selected_platform = user_input
        else:
            print(f"'{user_input}' is not a valid platform name. Please choose from the list above or enter 'exit'.")

    fetch_and_export_token_data(selected_platform)

if __name__ == "__main__":
    main()

Fetching available token platform names from CoinMarketCap API...

Available Token Platform Names:
  1. Algorand
  2. Aptos
  3. Arbitrum
  4. Arbitrum Nova
  5. Astar
  6. Atomicals-ARC20
  7. Avalanche C-Chain
  8. Avalanche DFK
  9. BNB Beacon Chain (BEP2)
  10. BNB Smart Chain (BEP20)
  11. Base
  12. Bera Chain
  13. Bitcichain
  14. Blast
  15. Canto
  16. Cardano
  17. Celo
  18. Chiliz Chain
  19. Chiliz Legacy Chain
  20. Core
  21. Cosmos
  22. Cronos
  23. Dogechain(EVM)
  24. Ethereum
  25. Etherlink
  26. Fantom
  27. Flare Network
  28. Fuse
  29. Gnosis Chain
  30. HECO
  31. Hedera Hashgraph
  32. HyperEVM
  33. Hyperliquid
  34. ICON
  35. ICP
  36. Injective
  37. KAIA
  38. Linea
  39. Manta Pacific
  40. Mantle
  41. Merlin
  42. Metal
  43. Metis Andromeda
  44. Moonbeam
  45. MultiversX
  46. Near
  47. Neo
  48. ONT
  49. ONUS
  50. Optimism
  51. Ordinals - BRC20
  52. Osmosis
  53. Polkadot
  54. Polygon
  55. PulseChain
  56. RSK RBTC
  57. Ronin
  58. Runes
 